## Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

## Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

model=init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000222F9A35A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000222F9A367B0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The Title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rarting out of 10")

In [4]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000222F9A35A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000222F9A367B0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The Title of the movie', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rarting out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [5]:
model.invoke("Provide the details about the movie Pushpa")

AIMessage(content='<think>\nOkay, so I need to find out the details about the movie Pushpa. Let me start by recalling what I know. Pushpa is a Telugu movie, right? I think it\'s a 2021 film. The director is Sukumar, who\'s known for some other Telugu films. The lead actor is Allu Arjun, who\'s a big name in Tollywood. The female lead is Rashmika Mandanna, I believe. The story is about a red sandalwood smuggler, maybe? There\'s a plot involving the illegal sandalwood trade in South India. \n\nWait, the title "Pushpa" refers to red sandalwood, which is a valuable and protected species. The protagonist is a smuggler who becomes a hero or something. There\'s also a part where he tries to take down the corrupt system involved in the smuggling. The movie might have a lot of action, maybe some heist elements. \n\nThe music is by Devi Sri Prasad, right? The songs are probably popular. The cinematography is by Ravi Varma, and the editing by Naveen Nooli. The supporting cast includes Fahadh Faas

In [7]:
model_with_structure.invoke("Provide the details about the movie Pushpa")

Movie(title='Pushpa', year=2021, director='Sukumar', rating=8.5)

### Message output alongside Parsed structure

In [8]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie, include_raw=True)  

response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for details about the movie Inception. Let me check the tools provided. There's a Movie function that requires title, year, director, and rating. I need to fill those parameters. I remember Inception was directed by Christopher Nolan. It came out in 2010. The rating is probably around 8.8 on IMDb. Let me confirm the year and director to be sure. Yep, 2010 and Christopher Nolan. The rating is 8.8. So I'll structure the tool call with those details.\n", 'tool_calls': [{'id': 'da820f308', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 231, 'total_tokens': 396, 'completion_time': 0.287480681, 'completion_tokens_details': {'reasoning_tokens': 117}, 'prompt_time': 0.009774311, 'prompt_tokens_details': None, 'queue_time': 

### Nested Structure


In [9]:
from pydantic import BaseModel,Field

class Actor(BaseModel):
    name:str
    role:str

class MovieDetails(BaseModel):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None,description="Budget in Indian rupees")
    

In [12]:
model_with_structure=model.with_structured_output(MovieDetails)

response=model_with_structure.invoke("Provide the details about the movie Pushpa-2")
response

MovieDetails(title='Pushpa-2', year='2024', cast=[Actor(name='Allu Arjun', role='Pushpa Raj'), Actor(name='Rashmika Mandanna', role='Rose'), Actor(name='Fahadh Faasil', role='Bhanwar Singh'), Actor(name='Avinash-Kumar', role='Inspector Jagan')], genres=['Action', 'Drama', 'Thriller'], budget=None)